<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Explanations/blob/main/notebooks/test/S2D_O600G4_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%time
! unzip -q drive/MyDrive/UNC/S2022P/GBIF_O_21_fourth.zip -d /content/temp
! mv /content/temp/content/dataset/test /content/test
! rm -rf /content/temp

CPU times: user 47.3 ms, sys: 24.9 ms, total: 72.3 ms
Wall time: 5.87 s


In [4]:
! cp drive/MyDrive/UNC/S2022P/annotations.txt /content/annotations.txt

In [5]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10841, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 10841 (delta 207), reused 264 (delta 160), pack-reused 10473
Receiving objects: 100% (10841/10841), 20.49 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (7922/7922), done.


In [6]:
! python -u ./pytorch-image-models/inference.py test -b 128 --input-size 3 600 600 --num-classes 185 --no-test-pool --checkpoint /content/drive/MyDrive/UNC/S2022P/output/20220816-163320-sequencer2d_l-600/last.pth.tar --model sequencer2d_l

Loaded state_dict from checkpoint '/content/drive/MyDrive/UNC/S2022P/output/20220816-163320-sequencer2d_l-600/last.pth.tar'
Model sequencer2d_l created, param count: 53984441
Predict: [0/7] Time 16.617 (16.617)


In [7]:
from glob import glob
import os
from collections import defaultdict

class_dict = {}
for fn in glob("test/**/*.*", recursive=True):
  cat = int(fn.split("/")[-2])
  bn = os.path.basename(fn)

  class_dict[bn] = cat

In [30]:
genera_dict = {}
species_dict = {}
anno_file = open("annotations.txt")
anno_lines = anno_file.readlines()

for l in anno_lines:
  l = l.strip().split(",")
  cat, species, genus = l[0], l[1], l[1].split(" ")[0]

  genera_dict[int(cat)] = genus
  species_dict[int(cat)] = species

In [9]:
print(genera_dict.items())

dict_items([(0, 'Aplectrum'), (1, 'Arethusa'), (2, 'Aspidogyne'), (3, 'Basiphyllaea'), (4, 'Beloglottis'), (5, 'Bletia'), (6, 'Bletia'), (7, 'Bletia'), (8, 'Bletilla'), (9, 'Brassia'), (10, 'Bulbophyllum'), (11, 'Calopogon'), (12, 'Calopogon'), (13, 'Calopogon'), (14, 'Calopogon'), (15, 'Calopogon'), (16, 'Calopogon'), (17, 'Calopogon'), (18, 'Campylocentrum'), (19, 'Cleistesiopsis'), (20, 'Cleistesiopsis'), (21, 'Cleistesiopsis'), (22, 'Corallorhiza'), (23, 'Corallorhiza'), (24, 'Corallorhiza'), (25, 'Corallorhiza'), (26, 'Corallorhiza'), (27, 'Corallorhiza'), (28, 'Corallorhiza'), (29, 'Cranichis'), (30, 'Cyclopogon'), (31, 'Cyclopogon'), (32, 'Cymbidium'), (33, 'Cypripedium'), (34, 'Cypripedium'), (35, 'Cypripedium'), (36, 'Cypripedium'), (37, 'Cypripedium'), (38, 'Cypripedium'), (39, 'Cypripedium'), (40, 'Cypripedium'), (41, 'Cyrtopodium'), (42, 'Cyrtopodium'), (43, 'Dactylorhiza'), (44, 'Dactylorhiza'), (45, 'Dactylorhiza'), (46, 'Dactylorhiza'), (47, 'Dendrophylax'), (48, 'Dendro

In [31]:
def get_acc():
  output_file = open("topk_ids.csv")
  output_lines = output_file.readlines()

  corr = 0
  total = 0
  for line in output_lines:
    
    line = line.strip().split(",")
    cat = class_dict[line[0]]
    pred = int(line[1])

    if cat == pred:
      corr += 1

    total += 1

  return corr / total

def sum_wrong(results_list):
  total = 0
  for v in results_list:
    if v == 0:
      total += 1

  return total

def acc_stats():
    species_wrong = defaultdict(list)
    genera_wrong = defaultdict(list)
    output_file = open("topk_ids.csv")
    output_lines = output_file.readlines()

    corr = 0
    total = 0
    for line in output_lines:
      line = line.strip().split(",")
      cat = class_dict[line[0]]
      gen = genera_dict[cat]
      pred = int(line[1])

      if cat == pred:
        corr += 1
        species_wrong[cat].append(1)
        genera_wrong[gen].append(1)
      else:
        species_wrong[cat].append(0)
        genera_wrong[gen].append(0)

    species_wrong = {k: sum_wrong(v) / len(v) for k, v in sorted(species_wrong.items(), key=lambda item: sum_wrong(item[1]) / len(item[1]), reverse=True)}
    genera_wrong = {k: sum_wrong(v) / len(v) for k, v in sorted(genera_wrong.items(), key=lambda item: sum_wrong(item[1]) / len(item[1]), reverse=True)}

    print([(species_dict.get(v[0]), v[1]) for v in list(species_wrong.items())[:20]])
    print(list(genera_wrong.items())[:5])

In [32]:
# S2D O600G4 Epoch 80/80
print(get_acc())

0.6254458977407847


In [33]:
acc_stats()

[('Bletia florida (Salisb.) R.Br.', 1.0), ('Corallorhiza maculata var. maculata', 1.0), ('Maxillaria crassifolia (Lindl.) Rchb.f.', 1.0), ('Platanthera cristata (Michx.) Lindl.', 1.0), ('Platanthera orbiculata var. orbiculata', 1.0), ('Prosthechea boothiana (Lindl.) W.E.Higgins', 1.0), ('Spiranthes brevilabris var. floridana (Wherry) Luer', 1.0), ('Spiranthes igniorchis M.C.Pace', 1.0), ('Spiranthes ovalis Lindl.', 1.0), ('Spiranthes Ã\x97kapnosperia M.C.Pace', 1.0), ('Triphora yucatanensis Ames', 1.0), ('Calopogon tuberosus var. tuberosus', 0.8), ('Corallorhiza maculata Raf.', 0.8), ('Corallorhiza odontorhiza (Willd.) Nutt.', 0.8), ('Cypripedium parviflorum var. pubescens (Willd.) O.W.Knight', 0.8), ('Epipactis helleborine subsp. helleborine', 0.8), ('Ionopsis utricularioides (Sw.) Lindl.', 0.8), ('Platanthera aquilonis Sheviak', 0.8), ('Platanthera blephariglottis (Willd.) Lindl.', 0.8), ('Platanthera ciliaris (L.) Lindl.', 0.8)]
[('Ionopsis', 0.8), ('Maxillaria', 0.75), ('Triphora',